## Dicom to Image

In [1]:
import cv2
from PIL import Image
import numpy as np
import os
import pandas as pd
import pydicom
import matplotlib.pyplot as plt

* csv : csv에서 Implant-Manufacturer을 Label로 사용

In [2]:
df = pd.read_csv('/workspace/20221129.csv')

/tmp/ipykernel_2211439/1487428822.py:1: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspace/20221129.csv')


In [3]:
df

,데이터명,Case_No.,ID,Sex,Age,Insertion date,Removal date,Side,Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4),Texture\n(Smooth: 0\nMicrotextured: 1\nTextured: 2),...,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379,Unnamed: 16380,Unnamed: 16381,Unnamed: 16382,Unnamed: 16383
0,NaN,1,28170976,F,34,6/1/2000,3/13/2007,L,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,28170976,F,34,6/1/2000,3/13/2007,L,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,28170976,F,34,6/1/2000,3/13/2007,L,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2,24915920,F,50,8/8/2007,NaN,L,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2,24915920,F,50,8/8/2007,NaN,L,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,NaN,896,44568032,F,46세,11/24/2017,NaN,L,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2185,NaN,897,48863878,F,60세,11/28/2017,NaN,R,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,NaN,898,50156319,F,66세,11/29/2017,NaN,L,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,NaN,899,49230565,F,52세,11/30/2017,NaN,R,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(set(df['ID']))

900

In [5]:
len(df)

2189

→ 701명 1978장의 데이터

In [6]:
df = df.dropna(axis=1)

In [7]:
len(df)

2189

In [8]:
df.columns

Index(['Case_No.', 'ID', 'Sex', 'Age', 'Insertion  date',
       'Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4)',
       'Texture\n(Smooth: 0\nMicrotextured: 1\nTextured: 2)',
       'Rupture\n(Intact: 0\nRuptured: 1)', 'sono date', 'Num_Img'],
      dtype='object')

In [9]:
df.head()

,Case_No.,ID,Sex,Age,Insertion date,Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4),Texture\n(Smooth: 0\nMicrotextured: 1\nTextured: 2),Rupture\n(Intact: 0\nRuptured: 1),sono date,Num_Img
0,1,28170976,F,34,6/1/2000,3,0,0,2001,4
1,1,28170976,F,34,6/1/2000,3,0,0,2003,1
2,1,28170976,F,34,6/1/2000,3,0,1,2007,3
3,2,24915920,F,50,8/8/2007,0,0,0,2008,5
4,2,24915920,F,50,8/8/2007,0,0,0,2009,2


* Exclusion = 1 제외

In [10]:
df['Exclusion여부'] = df['Rupture\n(Intact: 0\nRuptured: 1)'].astype('str')

/tmp/ipykernel_2211439/3905389438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Exclusion여부'] = df['Rupture\n(Intact: 0\nRuptured: 1)'].astype('str')


In [11]:
df['Exclusion여부'].unique()

array(['0', '1'], dtype=object)

In [12]:
(df['Exclusion여부'] == '0')

0        True
1        True
2       False
3        True
4        True
        ...  
2184     True
2185     True
2186     True
2187     True
2188     True
Name: Exclusion여부, Length: 2189, dtype: bool

In [13]:
(df['Exclusion여부'] == 'nan')

0       False
1       False
2       False
3       False
4       False
        ...  
2184    False
2185    False
2186    False
2187    False
2188    False
Name: Exclusion여부, Length: 2189, dtype: bool

In [14]:
df = df[(df['Exclusion여부'] == '0') | (df['Exclusion여부'] == 'nan')] ; df.head()

,Case_No.,ID,Sex,Age,Insertion date,Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4),Texture\n(Smooth: 0\nMicrotextured: 1\nTextured: 2),Rupture\n(Intact: 0\nRuptured: 1),sono date,Num_Img,Exclusion여부
0,1,28170976,F,34,6/1/2000,3,0,0,2001,4,0
1,1,28170976,F,34,6/1/2000,3,0,0,2003,1,0
3,2,24915920,F,50,8/8/2007,0,0,0,2008,5,0
4,2,24915920,F,50,8/8/2007,0,0,0,2009,2,0
5,2,24915920,F,50,8/8/2007,0,0,0,2011,2,0


In [15]:
len(df)

2186

In [16]:
len(df['ID'].unique())

898

→ 900명, 2189개 중 2명, 3개 제외

* 사용할 데이터만 남기기

In [38]:
data = df[1:][['ID', 'Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4)']] ; data

,ID,Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4)
1,28170976,3
3,24915920,0
4,24915920,0
5,24915920,0
6,24915920,0
...,...,...
2184,44568032,0
2185,48863878,2
2186,50156319,0
2187,49230565,2


In [40]:
data.rename(columns={'Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAllergan: 2\nSebbin: 3\nMotiva: 4)':'Num_Img'}, inplace=True)

In [41]:
data

,ID,Num_Img
1,28170976,3
3,24915920,0
4,24915920,0
5,24915920,0
6,24915920,0
...,...,...
2184,44568032,0
2185,48863878,2
2186,50156319,0
2187,49230565,2


* 중복 제거 : 중복 없음

In [42]:
data = data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [43]:
len(set(data['ID']))

898

* 데이터 환자수 확인

In [44]:
dicom_path = '/workspace/dataset_dec/media'

In [45]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [46]:
get_files_count(dicom_path)

900

* image save : dicom 이미지를 array → Image로 변환하여 저장<br>
이때 image의 환자명을 csv와 매칭하여 Implant 1번이 Label이 되게 한다.

In [47]:
img_path = '/workspace/dataset_dec/media'

* dcm to array : dataframe으로 만들어서 train_test_split 후 저장

In [48]:
dcm_to_arr =  pd.DataFrame()

In [49]:
t_ct = []
# 이미지로 변환 성공한 url
f_ct = []
# 이미지로 변환 실패한 url
dcm_all = []
# dicom 전체 url

## True df 구성 ##
img_arr = []
# img array
lb_arr = []
# label array
path_arr = []
# path array

## False df 구성 ##
#f_img_arr = []
# img array
f_lb_arr = []
# label array
#f_path_arr = []
# path array
for top, _, f in os.walk(img_path):
    # img_path에 접근하여 폴더별로 이미지에 접근한다.
    # top은 img_path/세부경로
    # f는 환자별 image file 이름
    for ix, filename in enumerate(f):
        # 그중 f는 각 환자 폴더 안의 이미지 전부의 주소이다.
        # filename은 그 중 하나에 접근.
        url = os.path.join(top,filename)
        # 세부 이미지 경로
        id = int(url.split('/')[4])
        # 환자 number
        dcm_all.append(url)
        break

In [50]:
dcm_all

['/workspace/dataset_dec/media/48207865/17_02_22_L_020.dcm',
 '/workspace/dataset_dec/media/52017201/21-01-15_L_1100003.dcm',
 '/workspace/dataset_dec/media/50676642/21-12-14_L_1200002.dcm',
 '/workspace/dataset_dec/media/32430451/22-03-17_R_1100001.dcm',
 '/workspace/dataset_dec/media/51870896/22_02_08_R_000002.dcm',
 '/workspace/dataset_dec/media/52957284/22-07-27_L_0000003.dcm',
 '/workspace/dataset_dec/media/53203858/21_01_14_L_000001.dcm',
 '/workspace/dataset_dec/media/31278775/211109_L_1100006.dcm',
 '/workspace/dataset_dec/media/41947249/21-11-24_L_0000002.dcm',
 '/workspace/dataset_dec/media/50145186/22_03_31_R_1100002.dcm',
 '/workspace/dataset_dec/media/47754964/22_04_18_R_0200001.dcm',
 '/workspace/dataset_dec/media/52698686/22-08-25_L_0000007.dcm',
 '/workspace/dataset_dec/media/49812051/20_05_25_L_1200001.dcm',
 '/workspace/dataset_dec/media/51162780/20-06-19_R_2200001.dcm',
 '/workspace/dataset_dec/media/54979064/220802_R_0000004.dcm',
 '/workspace/dataset_dec/media/4509

In [51]:
dcm = pydicom.dcmread('/workspace/dataset_dec/media/49368460/20_10_29_L_2200003.dcm')

In [52]:
img = dcm.pixel_array

In [53]:
t_ct = []
# 이미지로 변환 성공한 url
f_ct = []
# 이미지로 변환 실패한 url
dcm_all = []
# dicom 전체 url
t_id = []
## True df 구성 ##
img_arr = []
# img array
lb_arr = []
# label array
path_arr = []
# path array

## False df 구성 ##
#f_img_arr = []
# img array
f_lb_arr = []
# label array
#f_path_arr = []
# path array



for top, _, f in os.walk(img_path):
    # img_path에 접근하여 폴더별로 이미지에 접근한다.
    # top은 img_path/세부경로
    # f는 환자별 image file 이름
    for ix, filename in enumerate(f):
        # 그중 f는 각 환자 폴더 안의 이미지 전부의 주소이다.
        # filename은 그 중 하나에 접근.
        url = os.path.join(top,filename)
        # 세부 이미지 경로
        id = int(url.split('/')[4])
        # 환자 number
        dcm_all.append(url)
        # dicom 전체 url
        try:

            label = str(data[data['ID'] == id].iloc[0].Num_Img)
            # label을 csv의 Implant에 해당하는 값으로 지정
            dcm = pydicom.dcmread(url)
            # dcm read
            img = dcm.pixel_array
            # try 적용 영역 : 변환이 안되면 except 문으로
            lb_arr.append(label)
            # label 순서대로 더하기 : df에 사용
            img_arr.append(img)
            # array 순서대로 더하기 : df에 사용
            # dcm2array
            save_img = Image.fromarray(img)
            # array2img
            t_ct.append(url)
            t_id.append(id)
            # 성공적으로 읽어온 rul
            save_path = '/workspace/pre_dataset2/' + label
            # 이미지로 저장할 label url : label별로 저장
            img_save_path = save_path + '/' + url.split('/')[-2]+ str(ix) + '.jpg'
            path_arr.append(img_save_path)
            # 이미지 세부 경로 순서대로 더하기 : df에 사용
            # label url에 세부적으로 저장할 img url, 중복된 이미지명 존재로 인해 index도 이름에 같이 넣어준다.
          #  if not os.path.isdir(save_path):
           #     os.makedirs(save_path)
                # 폴더가 없으면 생성
           # save_img.save(img_save_path)
            # 이미지 저장

        except:
            f_ct.append(url)
            try:
                f_lb_arr.append(label)
            except:
                f_lb_arr.append('NaN')
            # dicom으로 읽지 못한 url 저장

            print('error: ' , url)

error:  /workspace/dataset_dec/media/29473483/04_04_01_R_3200002.dcm
error:  /workspace/dataset_dec/media/29473483/04_04_01_R_3200001.dcm
error:  /workspace/dataset_dec/media/22848466/18_11_17_R_011002.dcm
error:  /workspace/dataset_dec/media/22848466/18_11_17_R_011001.dcm
error:  /workspace/dataset_dec/media/49479663/2202250002.dcm
error:  /workspace/dataset_dec/media/49479663/2202250003.dcm
error:  /workspace/dataset_dec/media/49479663/2202250001.dcm
error:  /workspace/dataset_dec/media/49479663/200804.dcm
error:  /workspace/dataset_dec/media/27804531/09_10_28_R_411.dcm
error:  /workspace/dataset_dec/media/28170976/03_04_02_L_0020001.dcm
error:  /workspace/dataset_dec/media/28170976/01_02_21_L_0020003.dcm
error:  /workspace/dataset_dec/media/28170976/01_02_21_L_0020002.dcm
error:  /workspace/dataset_dec/media/28170976/01_02_21_L_0020004.dcm
error:  /workspace/dataset_dec/media/28170976/01_02_21_L_0020001.dcm


* 예외처리된 링크는 dcmread로 읽히지 않음

In [54]:
# dcm = pydicom.dcmread('/workspace/dataset_dec/media/28170976/01_02_21_L_0020001.dcm')

In [55]:
#img = dcm.pixel_array

In [57]:
len(t_ct)

5882

In [58]:
len(f_ct)

14

* 읽힌 파일과 읽지 못한 파일

In [59]:
len(dcm_all)

5896

In [60]:
len(t_ct)

5882

In [61]:
len(f_ct)

14

* jpg check

In [62]:
ecp = []
for pt in f_ct:
    if pt.split('.')[-1] == 'jpg':
        pass
    else:
        ecp.append(pt)

In [63]:
ecp

['/workspace/dataset_dec/media/29473483/04_04_01_R_3200002.dcm',
 '/workspace/dataset_dec/media/29473483/04_04_01_R_3200001.dcm',
 '/workspace/dataset_dec/media/22848466/18_11_17_R_011002.dcm',
 '/workspace/dataset_dec/media/22848466/18_11_17_R_011001.dcm',
 '/workspace/dataset_dec/media/49479663/2202250002.dcm',
 '/workspace/dataset_dec/media/49479663/2202250003.dcm',
 '/workspace/dataset_dec/media/49479663/2202250001.dcm',
 '/workspace/dataset_dec/media/49479663/200804.dcm',
 '/workspace/dataset_dec/media/27804531/09_10_28_R_411.dcm',
 '/workspace/dataset_dec/media/28170976/03_04_02_L_0020001.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020003.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020002.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020004.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020001.dcm']

In [64]:
ct_f_num = []
for i in f_ct:
    ct_f_num.append(i.split('/')[4])

In [65]:
set(ct_f_num)

{'22848466', '27804531', '28170976', '29473483', '49479663'}

In [66]:
len(set(ct_f_num))

5

In [67]:
len(set(t_id))

897

* dcmread로 읽혀지지 않는 파일 14개
→ 897명 5882장으로 진행

* data frame 구성

In [68]:
df = pd.DataFrame([ x for x in zip(t_ct,lb_arr, t_id)], columns = ['url', 'label', 'id'])

In [69]:
df.head()

,url,label,id
0,/workspace/dataset_dec/media/48207865/17_02_22...,0,48207865
1,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
2,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
3,/workspace/dataset_dec/media/52017201/22-03-21...,1,52017201
4,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201


In [70]:
df.to_csv('df.csv', index=False)

In [71]:
len(df['id'].unique())

897

* 읽지 못한 url

In [72]:
1296+2062+1911+1238+594+289

7390

In [73]:
347+433+412+247+127+66


1632

In [74]:
259+430+361+265+110+85

1510

In [75]:
f_ct

['/workspace/dataset_dec/media/29473483/04_04_01_R_3200002.dcm',
 '/workspace/dataset_dec/media/29473483/04_04_01_R_3200001.dcm',
 '/workspace/dataset_dec/media/22848466/18_11_17_R_011002.dcm',
 '/workspace/dataset_dec/media/22848466/18_11_17_R_011001.dcm',
 '/workspace/dataset_dec/media/49479663/2202250002.dcm',
 '/workspace/dataset_dec/media/49479663/2202250003.dcm',
 '/workspace/dataset_dec/media/49479663/2202250001.dcm',
 '/workspace/dataset_dec/media/49479663/200804.dcm',
 '/workspace/dataset_dec/media/27804531/09_10_28_R_411.dcm',
 '/workspace/dataset_dec/media/28170976/03_04_02_L_0020001.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020003.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020002.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020004.dcm',
 '/workspace/dataset_dec/media/28170976/01_02_21_L_0020001.dcm']

In [76]:
#f_df = pd.DataFrame([ x for x in zip(f_ct, f_lb_arr)], columns = ['url', 'check'])

In [77]:
#f_df.to_csv('f_df.csv', index=False)

* data leakage 방지 split

In [78]:
df.head()

,url,label,id
0,/workspace/dataset_dec/media/48207865/17_02_22...,0,48207865
1,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
2,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
3,/workspace/dataset_dec/media/52017201/22-03-21...,1,52017201
4,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201


→ 동일한 id의 파일은 train, valid, test중 모두 하나에 속해야함.

In [79]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv('/workspace/Pipeline_version3_manufacturer/1_data_preprocessing/df.csv')

In [82]:
df

,url,label,id
0,/workspace/dataset_dec/media/48207865/17_02_22...,0,48207865
1,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
2,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
3,/workspace/dataset_dec/media/52017201/22-03-21...,1,52017201
4,/workspace/dataset_dec/media/52017201/21-01-15...,1,52017201
...,...,...,...
5877,/workspace/dataset_dec/media/49368460/22_03_26...,2,49368460
5878,/workspace/dataset_dec/media/49368460/22_03_26...,2,49368460
5879,/workspace/dataset_dec/media/49368460/20_10_29...,2,49368460
5880,/workspace/dataset_dec/media/49368460/22_03_26...,2,49368460


In [83]:
len(set(list(df['id'])))

897

* train valid split

In [84]:
df['id']

0       48207865
1       52017201
2       52017201
3       52017201
4       52017201
          ...   
5877    49368460
5878    49368460
5879    49368460
5880    49368460
5881    49368460
Name: id, Length: 5882, dtype: int64

In [89]:
df_du = df[['label', 'id']].drop_duplicates('id').reindex()

In [90]:
df_du

,label,id
0,0,48207865
1,1,52017201
7,1,50676642
12,1,32430451
15,0,51870896
...,...,...
5850,2,50466489
5852,0,53604398
5853,1,50205451
5866,1,34716454


In [91]:
df_du['label'].value_counts()

0    528
1    260
2     91
3     18
Name: label, dtype: int64

In [92]:
df_du = df_du[df_du['label'].map(lambda x : x in [0, 1, 2, 3])]

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
x_train, x_valid ,y_train, y_valid = train_test_split(df_du['id'], df_du['label'], test_size=0.3, shuffle=True, stratify=df_du['label'], random_state=2022)

In [95]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

(627,) (627,) (270,) (270,)


* valid test split

In [96]:
x_valid, x_test ,y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, shuffle=True, stratify=y_valid, random_state=2022)

In [97]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(627,) (627,) (135,) (135,) (135,) (135,)


In [98]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(627,) (627,) (135,) (135,) (135,) (135,)


In [99]:
import collections

In [100]:
collections.Counter(y_train)

Counter({1: 182, 0: 369, 2: 64, 3: 12})

In [101]:
collections.Counter(y_valid)

Counter({0: 80, 1: 39, 2: 13, 3: 3})

In [102]:
collections.Counter(y_test)

Counter({0: 79, 1: 39, 3: 3, 2: 14})

* train save

In [115]:
df_x_train = df[df['id'].map(lambda x : x in list(x_train))]

In [116]:
len(df_x_train)

4154

In [117]:
for idx, (label, path) in enumerate(zip(df_x_train['label'], df_x_train['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    label = label
    fd_path = '/workspace/pre_dataset2/train/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* valid save

In [118]:
df_x_valid = df[df['id'].map(lambda x : x in list(x_valid))]

In [119]:
len(df_x_valid)

909

In [120]:
for idx, (label, path) in enumerate(zip(df_x_valid['label'], df_x_valid['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    label = label
    fd_path = '/workspace/pre_dataset2/valid/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* test save

In [121]:
df_x_test = df[df['id'].map(lambda x : x in list(x_test))]

In [122]:
len(df_x_test)

819

In [123]:
for idx, (label, path) in enumerate(zip(df_x_test['label'], df_x_test['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    label = label
    fd_path = '/workspace/pre_dataset2/test/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* 갯수 확인

In [124]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [126]:
train_0 = get_files_count('/workspace/pre_dataset2/train/0')
train_1 = get_files_count('/workspace/pre_dataset2/train/1')
train_2 = get_files_count('/workspace/pre_dataset2/train/2')
train_3 = get_files_count('/workspace/pre_dataset2/train/3')

In [127]:
print(train_0, train_1, train_2, train_3)

2418 1304 384 48


In [128]:
2418 +1304+ 384+ 48

4154

In [129]:
valid_0 = get_files_count('/workspace/pre_dataset2/valid/0')
valid_1 = get_files_count('/workspace/pre_dataset2/valid/1')
valid_2 = get_files_count('/workspace/pre_dataset2/valid/2')
valid_3 = get_files_count('/workspace/pre_dataset2/valid/3')

In [130]:
print(valid_0, valid_1, valid_2, valid_3)

542 276 79 12


In [131]:
542 +276+ 79+ 12

909

In [132]:
test_0 = get_files_count('/workspace/pre_dataset2/test/0')
test_1 = get_files_count('/workspace/pre_dataset2/test/1')
test_2 = get_files_count('/workspace/pre_dataset2/test/2')
test_3 = get_files_count('/workspace/pre_dataset2/test/3')

In [133]:
print(test_0, test_1, test_2, test_3)

428 289 85 17


In [134]:
428 +289+ 85+ 17

819

In [135]:
4019 + 873 + 849

5741

→ 갯수 이슈 없음